In [1]:
import gym
import numpy as np
import pandas as pd
import time
import datetime
import random

In [2]:
%%time
ROWS = None
ROWS = 100000

df = pd.read_csv('data/train.csv.gz', 
                        nrows=ROWS,
                        compression='gzip',
                        error_bad_lines=False)

CPU times: user 671 ms, sys: 56 ms, total: 727 ms
Wall time: 725 ms


In [3]:
clusters = list(range(5))
df = df[df.hotel_cluster.isin(clusters)].reset_index()
df.shape

(4853, 25)

# Q-learning
***

In [4]:
def init_q(s, a, type="ones"):
    """
    @param s the number of states
    @param a the number of actions
    @param type random, ones or zeros for the initialization
    """
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))

In [5]:
def epsilon_greedy(Q, epsilon, n_actions, s, train=False):
    """
    @param Q Q values state x action -> value
    @param epsilon for exploration
    @param s number of states
    @param train if true then no random actions selected
    """
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action

In [6]:
def step(df, idx, action):
    observation = df.iloc[idx]['hotel_cluster']
    reward = -1
    
    if(observation == action):
        reward = 1
    
    return reward

In [17]:
# def qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, render = False, test=False):
"""
@param alpha learning rate
@param gamma decay factor
@param epsilon for exploration
@param max_steps for max step in each episode
@param n_tests number of test episodes
"""
#===PARAMS===#
alpha = 0.4
gamma = 0.999
epsilon = 0.8
episodes = 10#10000
max_steps = 50#2500
n_tests = 2
#=============#


env = gym.make('Taxi-v2')
n_states = df.shape[0]
actions = np.sort(df.hotel_cluster.unique()) #hotel_clusters
n_actions = actions.size

Q = init_q(n_states, n_actions, type="zeros")


timestep_reward = []

for idx, row in df.iterrows():
    s = idx
    a = epsilon_greedy(Q, epsilon, n_actions, s)
    t = 0
    total_reward = 0
    done = False #check if it is the last row 
    
    if (idx % 100 == 0):
        print(idx)
    
    
    while t < max_steps:
        t += 1
        reward = step(df, idx, a)
        
        total_reward += reward

        done = (s == df.shape[0]-1)
        
        if done:
            Q[s, a] += alpha * ( reward  - Q[s, a] )
        else:
            s_ = s + 1
            a_ = np.argmax(Q[s_, :])
            Q[s, a] += alpha * ( reward + (gamma * Q[s_, a_]) - Q[s, a] )
        s, a = s_, a_
Q      

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800


array([[-0.4       ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.4       ,  0.4       ,  0.        ,  0.        ,  0.        ],
       [-0.4       ,  0.79984   ,  0.        ,  0.        ,  0.        ],
       ...,
       [-1.45249369, -1.48742386, -1.68192312,  0.209811  , -1.57908836],
       [-1.38969725, -1.56971863, -1.56785915, -0.1065511 , -1.55550692],
       [-1.        , -1.        , -1.        , -1.        , -1.        ]])

In [18]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[-0.4         0.          0.          0.          0.        ]
 [-0.4         0.4         0.          0.          0.        ]
 [-0.4         0.79984     0.          0.          0.        ]
 ...
 [-1.45249369 -1.48742386 -1.68192312  0.209811   -1.57908836]
 [-1.38969725 -1.56971863 -1.56785915 -0.1065511  -1.55550692]
 [-1.         -1.         -1.         -1.         -1.        ]]


# Examples
***

In [ ]:

import time

"""
Qlearning is an off policy learning python implementation.
This is a python implementation of the qlearning algorithm in the Sutton and
Barto's book on RL. It's called SARSA because - (state, action, reward, state,
action). The only difference between SARSA and Qlearning is that SARSA takes the
next action based on the current policy while qlearning takes the action with
maximum utility of next state.
Using the simplest gym environment for brevity: https://gym.openai.com/envs/FrozenLake-v0/
"""

def init_q(s, a, type="ones"):
    """
    @param s the number of states
    @param a the number of actions
    @param type random, ones or zeros for the initialization
    """
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))


def epsilon_greedy(Q, epsilon, n_actions, s, train=False):
    """
    @param Q Q values state x action -> value
    @param epsilon for exploration
    @param s number of states
    @param train if true then no random actions selected
    """
    if train or np.random.rand() < epsilon:
        action = np.argmax(Q[s, :])
    else:
        action = np.random.randint(0, n_actions)
    return action

def qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, render = False, test=False):
    """
    @param alpha learning rate
    @param gamma decay factor
    @param epsilon for exploration
    @param max_steps for max step in each episode
    @param n_tests number of test episodes
    """
    env = gym.make('Taxi-v2')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        print(f"Episode: {episode}")
        s = env.reset()
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        t = 0
        total_reward = 0
        done = False
        while t < max_steps:
            if render:
                env.render()
            t += 1
            s_, reward, done, info = env.step(a)
            total_reward += reward
            a_ = np.argmax(Q[s_, :])
            if done:
                Q[s, a] += alpha * ( reward  - Q[s, a] )
            else:
                Q[s, a] += alpha * ( reward + (gamma * Q[s_, a_]) - Q[s, a] )
            s, a = s_, a_
            if done:
                if render:
                    print(f"This episode took {t} timesteps and reward: {total_reward}")
                timestep_reward.append(total_reward)
                break
    if render:
        print(f"Here are the Q values:\n{Q}\nTesting now:")
    if test:
        test_agent(Q, env, n_tests, n_actions)
    return timestep_reward

def test_agent(Q, env, n_tests, n_actions, delay=1):
    for test in range(n_tests):
        print(f"Test #{test}")
        s = env.reset()
        done = False
        epsilon = 0
        while True:
            time.sleep(delay)
            env.render()
            a = epsilon_greedy(Q, epsilon, n_actions, s, train=True)
            print(f"Chose action {a} for state {s}")
            s, reward, done, info = env.step(a)
            if done:
                if reward > 0:
                    print("Reached goal!")
                else:
                    print("Shit! dead x_x")
                time.sleep(3)
                break


if __name__ =="__main__":
    alpha = 0.4
    gamma = 0.999
    epsilon = 0.9
    episodes = 10000
    max_steps = 2500
    n_tests = 2
    timestep_reward = qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, test = True)
    print(timestep_reward)

In [30]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .8
y = .95
num_episodes = 2000 #size of the dataset
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)

In [8]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.5425


In [9]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[1.99621167e-01 5.89175006e-03 8.07100994e-03 6.04850852e-03]
 [4.18034281e-03 2.74940669e-03 1.43662286e-06 1.86893913e-01]
 [6.03207088e-04 1.70233402e-03 7.81149229e-03 1.94338505e-01]
 [4.71433751e-04 6.44633989e-04 4.89921832e-04 5.12625796e-02]
 [2.08005473e-01 1.54144113e-04 2.39960635e-03 3.57595370e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.07535346e-05 3.71499356e-05 1.04479335e-01 6.62555811e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.75994800e-03 0.00000000e+00 1.02259355e-03 1.99179271e-01]
 [0.00000000e+00 5.22050616e-01 0.00000000e+00 0.00000000e+00]
 [1.40918714e-01 0.00000000e+00 7.38060901e-04 1.50394224e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.47850461e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.58250879e-01]
 [0.00000000e+00 0.00000000e+00 0.